## Day 19: Go With The Flow

https://adventofcode.com/2018/day/19

### Part 1

Functions copied from [Day 16](https://github.com/mratford/advent/blob/master/2018/16/Chronal%20Classification.ipynb).

In [20]:
from pyrsistent import pvector, pmap, pset
import operator as op
from parse import parse


def opr(f):
    def fr(registers, a, b, c):
        return registers.set(c, f(registers[a], registers[b]))
    
    return fr


def opi(f):
    def fi(registers, a, b, c):
        return registers.set(c, f(registers[a], b))
    
    return fi


addr = opr(op.add)
addi = opi(op.add)
mulr = opr(op.mul)
muli = opi(op.mul)

banr = opr(op.and_)
bani = opi(op.and_)
borr = opr(op.or_)
bori = opi(op.or_)

def setr(r, a, b, c):
    return r.set(c, r[a])
def seti(r, a, b, c):
    return r.set(c, a)

def gtir(r, a, b, c):
    return r.set(c, 1 if a > r[b] else 0)
def gtri(r, a, b, c):
    return r.set(c, 1 if r[a] > b else 0)
def gtrr(r, a, b, c):
    return r.set(c, 1 if r[a] > r[b] else 0)

def eqir(r, a, b, c):
    return r.set(c, 1 if a == r[b] else 0)
def eqri(r, a, b, c):
    return r.set(c, 1 if r[a] == b else 0)
def eqrr(r, a, b, c):
    return r.set(c, 1 if r[a] == r[b] else 0)

I'm not sure if I've ever used `eval` before.

In [21]:
opstrings = 'addr addi mulr muli banr bani borr bori setr seti gtir gtrr eqir eqri eqrr'.split(' ')
opcodes = pmap({op: eval(op) for op in opstrings})

In [22]:
def parse_program(program_data):
    lines = program_data.rstrip().splitlines()
    program = pvector()
    ip = parse('#ip {ip:d}', lines[0])['ip']
    
    for line in lines[1:]:
        opstring, a, b, c = parse('{} {:d} {:d} {:d}', line)
        program = program.append((opcodes[opstring], a, b, c))
        
    return ip, program


def run(ip, registers, program, debug=False):
    while True:
        if registers[ip] < 0 or registers[ip] >= len(program):
            return registers.set(ip, registers[ip] - 1)

        opcode, a, b, c = program[registers[ip]]
        registers = opcode(registers, a, b, c)
        registers = registers.set(ip, registers[ip] + 1)
        
        if debug:
            print(registers)

In [23]:
test_data = '''#ip 0
seti 5 0 1
seti 6 0 2
addi 0 1 0
addr 1 2 3
setr 1 0 0
seti 8 0 4
seti 9 0 5'''
test_ip, test_program = parse_program(test_data)

initial_registers = pvector([0, 0, 0, 0, 0, 0])
run(test_ip, initial_registers, test_program, debug=True)

pvector([1, 5, 0, 0, 0, 0])
pvector([2, 5, 6, 0, 0, 0])
pvector([4, 5, 6, 0, 0, 0])
pvector([6, 5, 6, 0, 0, 0])
pvector([7, 5, 6, 0, 0, 9])


pvector([6, 5, 6, 0, 0, 9])

In [24]:
data = open('input', 'r').read()
ip, program = parse_program(data)

In [25]:
%time run(ip, initial_registers, program)

CPU times: user 7.55 s, sys: 0 ns, total: 7.55 s
Wall time: 7.55 s


pvector([1500, 998, 999, 1, 999, 256])

1500 is the answer.

### Part 2

This is probably going to take a long time, but I'm off to the pub. If it hasn't finished by tomorrow morning then I'll rethink my approach.

In [47]:
# %time run(ip, initial_registers.set(0, 1), program)

KeyboardInterrupt: 

That hasn't stopped, let's have a look at what it's doing. First add line numbers so we can see what the instruction pointer is pointing at.

In [51]:
for i, line in enumerate(open('input').readlines()[1:]):
    print(f'# {i:02d} {line.rstrip()}')

# 00 addi 5 16 5
# 01 seti 1 1 4
# 02 seti 1 8 2
# 03 mulr 4 2 3
# 04 eqrr 3 1 3
# 05 addr 3 5 5
# 06 addi 5 1 5
# 07 addr 4 0 0
# 08 addi 2 1 2
# 09 gtrr 2 1 3
# 10 addr 5 3 5
# 11 seti 2 6 5
# 12 addi 4 1 4
# 13 gtrr 4 1 3
# 14 addr 3 5 5
# 15 seti 1 4 5
# 16 mulr 5 5 5
# 17 addi 1 2 1
# 18 mulr 1 1 1
# 19 mulr 5 1 1
# 20 muli 1 11 1
# 21 addi 3 7 3
# 22 mulr 3 5 3
# 23 addi 3 8 3
# 24 addr 1 3 1
# 25 addr 5 0 5
# 26 seti 0 9 5
# 27 setr 5 8 3
# 28 mulr 3 5 3
# 29 addr 5 3 3
# 30 mulr 5 3 3
# 31 muli 3 14 3
# 32 mulr 3 5 3
# 33 addr 1 3 1
# 34 seti 0 4 0
# 35 seti 0 3 5


Now translate it to python by hand. This doesn't do anything as the control flow isn't working.

In [52]:
r = [1, 0, 0, 0, 0, 0]

# The instruction pointer is stored in register 5
# I'll use a variable ip so it's clear where the
# control flow is

ip = 0

# 00 addi 5 16 5
ip = ip + 16 # GOTO 17
# 01 seti 1 1 4
r[4] = 1
# 02 seti 1 8 2
r[2] = 1
# 03 mulr 4 2 3
r[3] = r[4] * r[2]
# 04 eqrr 3 1 3
if r[3] == r[1]:
    r[3] = 1
else:
    r[3] = 0
# 05 addr 3 5 5
ip = ip + r[3]
# 06 addi 5 1 5
ip = ip + 1 # GOTO 8
# 07 addr 4 0 0
r[0] = r[4] + r[0]
# 08 addi 2 1 2
r[2] = r[2] + 1
# 09 gtrr 2 1 3
if r[2] > r[1]:
    r[3] = 1
else:
    r[3] = 0
# 10 addr 5 3 5
ip = ip + r[3] 
# 11 seti 2 6 5
ip = 2
# 12 addi 4 1 4
r[4] = r[4] + 1
# 13 gtrr 4 1 3
if r[4] > r[1]:
    r[3] = 1
else:
    r[3] = 0
# 14 addr 3 5 5
ip = ip + r[3] # GOTO ?
# 15 seti 1 4 5
ip = 1 # GOTO 1
# 16 mulr 5 5 5
ip = ip * ip # ????
# 17 addi 1 2 1
r[1] = r[1] + 2
# 18 mulr 1 1 1
r[1] = r[1] * r[1]
# 19 mulr 5 1 1
r[1] = ip * r[1]
# 20 muli 1 11 1
r[1] = r[1] * 11
# 21 addi 3 7 3
r[3] = r[3] + 7
# 22 mulr 3 5 3
r[3] = r[3] * ip
# 23 addi 3 8 3
r[3] = r[3] + 8
# 24 addr 1 3 1
r[1] = r[3] + r[1]
# 25 addr 5 0 5
ip = ip + r[0]
# 26 seti 0 9 5
ip = 0 # GOTO 0
# 27 setr 5 8 3
r[3] = ip
# 28 mulr 3 5 3
r[3] = r[3] * ip
# 29 addr 5 3 3
r[3] = r[3] + ip
# 30 mulr 5 3 3
r[3] = r[3] * ip
# 31 muli 3 14 3
r[3] = r[3] * 14
# 32 mulr 3 5 3
r[3] = r[3] * ip
# 33 addr 1 3 1
r[1] = r[1] + r[3]
# 34 seti 0 4 0
r[0] = 0
# 35 seti 0 3 5
ip = 0 # GOTO 1

Remove `ip` as appropriate, replacing with the line number in equations or working out the control flow if it's being set.

```
r = [1, 0, 0, 0, 0, 0]

# 01
for r4 in range(1, r[1] + 1)
    for r2 in range(1, r[1] + 1):
        r[3] = r4 * r2
    # 04 eqrr 3 1 3
        if r[3] == r[1]:
    # 07 addr 4 0 0
            r[0] = r[0] + r4
# 16 mulr 5 5 5
# GOTO 256 i.e. end

# START HERE
# 17-20
r[1] = ((r[1] + 2) ** 2) * 19 * 11
# 21-24
r[3] = (r[3] + 7) * 22 + 8 
r[1] = r[1] + r[3]
if r[0] = 0:
    # GOTO 1


# 27 setr 5 8 3
r[3] = 27
# 28 mulr 3 5 3
r[3] = r[3] * 28
# 29 addr 5 3 3
r[3] = r[3] + 29
# 30 mulr 5 3 3
r[3] = r[3] * 30
# 31 muli 3 14 3
r[3] = r[3] * 14
# 32 mulr 3 5 3
r[3] = r[3] * 32
# 33 addr 1 3 1
r[1] = r[1] + r[3]
# 34 seti 0 4 0
r[0] = 0
# 35 seti 0 3 5
# GOTO 1
```

The program starts at line 17 and sets registers 1 and 3, in this case to `4 * 19 * 11` and `7 * 22 + 8 + 4 * 19 * 11`. Then if register 0 is 0 the program jumps to the loop at line 1 and then exits, otherwise it performs more calculations on registers 1 and 3 to make them larger numbers before jumping to the loop. So restructuring the program as a function on the initial value in register 0, returning the final value in register 0, gives

In [16]:
def run_2(r0):
    r1, r2, r3, r4, r5 = 0, 0, 0, 0, 0

    r1 = ((r1 + 2) ** 2) * 19 * 11
    # 21-24
    r3 = (r3 + 7) * 22 + 8
    r1 = r1 + r3
    
    if r0 != 0:
        # 27 setr 5 8 3
        r3 = 27
        # 28 mulr 3 5 3
        r3 = r3 * 28
        # 29 addr 5 3 3
        r3 = r3 + 29
        # 30 mulr 5 3 3
        r3 = r3 * 30
        # 31 muli 3 14 3
        r3 = r3 * 14
        # 32 mulr 3 5 3
        r3 = r3 * 32
        # 33 addr 1 3 1
        r1 = r1 + r3
        # 34 seti 0 4 0
        r0 = 0
        # 35 seti 0 3 5

    for r4 in range(1, r1 + 1):
        for r2 in range(1, r1 + 1):
            r3 = r4 * r2
            
            if r3 == r1:
                r0 = r0 + r4
    
    return r0


assert run_2(0) == 1500

The final value of r0 is the sum of all numbers which are a factor, not necessarily prime, of r1. (Wasn't there one last year that was pretty much the same but with jumps rather than instruction pointers?) Let's try to speed the code up a bit.

In [28]:
import math

def run_2(r0):
    r1, r2, r3, r4, r5 = 0, 0, 0, 0, 0

    r1 = ((r1 + 2) ** 2) * 19 * 11
    # 21-24
    r3 = (r3 + 7) * 22 + 8
    r1 = r1 + r3
    
    if r0 != 0:
        # 27 setr 5 8 3
        r3 = 27
        # 28 mulr 3 5 3
        r3 = r3 * 28
        # 29 addr 5 3 3
        r3 = r3 + 29
        # 30 mulr 5 3 3
        r3 = r3 * 30
        # 31 muli 3 14 3
        r3 = r3 * 14
        # 32 mulr 3 5 3
        r3 = r3 * 32
        # 33 addr 1 3 1
        r1 = r1 + r3
        # 34 seti 0 4 0
        r0 = 0
        # 35 seti 0 3 5

    for r4 in range(1, r1 + 1):
        if r1 % r4 == 0:
            r0 = r0 + r4
    
    return r0
                    
                    
%time run_2(0)

CPU times: user 56 µs, sys: 1 µs, total: 57 µs
Wall time: 58.4 µs


1500

That's down from 7.5 seconds so it looks hopeful. Try with register 0 set to 1.

In [29]:
%time run_2(1)

CPU times: user 598 ms, sys: 0 ns, total: 598 ms
Wall time: 597 ms


18869760